Notebook with system commands to process FastQ files: mapping with Bowtie2, sorting and indexing with samtools, conversion to BigWigs with bamCoverage

In [1]:
# list all files ending in q.gz
!ls -lh *q.gz

-rw-r--r-- 1 boycem4 GEU33025 53M Nov 25  2020 ctrl_R1.fastq.gz
-rw-r--r-- 1 boycem4 GEU33025 53M Nov 30  2020 ctrl_R2.fastq.gz


In [2]:
# settings for subsequent commands

# select the sample to be processed:
sample = 'ctrl'

# prefix for output files can be the same as sample name
out_prefix = sample

# reference genome
reference = 'C_glabrata.fa'

# name for bowtie2 index files
bowtie_index = 'C_glabrata'

# number of threads to use in parallel
threads = 1

# execute stepwise or as a pipeline
#processing = 'stepwise'
processing = 'pipeline'

In [3]:
import os.path

# make sure the mapping index files are in place
for i in range(1,5) :
    bowtie_index_file = f'{bowtie_index}.{i}.bt2'

    if not os.path.isfile(bowtie_index_file) :
        
        print(f'file {bowtie_index_file} does not exist')
        
        # try to generate the index
        job = f'bowtie2-build {reference} {bowtie_index}'
        print(f'job: {job}')
        !time {job}
        
        break

file C_glabrata.1.bt2 does not exist
job: bowtie2-build C_glabrata.fa C_glabrata
Settings:
  Output files: "C_glabrata.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  C_glabrata.fa
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 3084501
Using parameters --bmax 2313376 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Construc

In [4]:
# Define the input file(s)
# (for paired-end reads we have two input files):
infile1 = f'{sample}_R1.fastq.gz'
infile2 = f'{sample}_R2.fastq.gz'

# one could also use trimmed files if available:
#infile1 = f'{sample}_R1_val_1.fq.gz'
#infile2 = f'{sample}_R2_val_2.fq.gz'

# make sure input files are in place
for file in (infile1, infile2) :
    if not os.path.isfile(file) :
        raise Exception(f'file {file} does not exist')

In [5]:
if processing == 'stepwise' :
    
    # run bowtie2 mapper and save output in SAM format
    job = f'bowtie2 -x {bowtie_index} -1 {infile1} -2 {infile2} --mm -p {threads} > {out_prefix}.sam'
    print(f'job: {job}')
    !time {job}
    
    # convert SAM into BAM format, so that it can be sorted
    job = f'samtools view -b {out_prefix}.sam > {out_prefix}.bam'
    print(f'job: {job}')
    !time {job}
    
    # sort BAM file (requires BAM file as input)
    job = f'samtools sort -@ {threads} {out_prefix}.bam > {out_prefix}.sorted.bam'
    print(f'job: {job}')
    !time {job}
    
    # replace unsorted BAM file with sorted one
    job = f'mv {out_prefix}.sorted.bam {out_prefix}.bam'
    print(f'job: {job}')
    !time {job}
    
else :
    
    # run bowtie2 mapper and save compressed and sorted output in {out_prefix}.bam in one go
    job = f'bowtie2 -x {bowtie_index} -1 {infile1} -2 {infile2} --mm -p {threads} | samtools view -b | samtools sort -@ {threads} > {out_prefix}.bam'
    print(f'job: {job}')
    !time {job}

job: bowtie2 -x C_glabrata -1 ctrl_R1.fastq.gz -2 ctrl_R2.fastq.gz --mm -p 1 | samtools view -b | samtools sort -@ 1 > ctrl.bam
1071087 reads; of these:
  1071087 (100.00%) were paired; of these:
    304520 (28.43%) aligned concordantly 0 times
    669889 (62.54%) aligned concordantly exactly 1 time
    96678 (9.03%) aligned concordantly >1 times
    ----
    304520 pairs aligned concordantly 0 times; of these:
      50408 (16.55%) aligned discordantly 1 time
    ----
    254112 pairs aligned 0 times concordantly or discordantly; of these:
      508224 mates make up the pairs; of these:
        468853 (92.25%) aligned 0 times
        23234 (4.57%) aligned exactly 1 time
        16137 (3.18%) aligned >1 times
78.11% overall alignment rate

real	2m3.030s
user	2m48.292s
sys	0m2.367s


In [6]:
# list all the SAM and BAM files in the directory
!ls -lh *.bam *.sam

ls: cannot access '*.sam': No such file or directory
-rw-r--r-- 1 boycem4 GEU3302521 3.7M Dec  1 19:46  chipA.bam
-rw-r--r-- 1 boycem4 GEU3302521  85M Dec  3 11:25  ctrl.bam


In [7]:
# create an index for the BAM file to allow direct access at certain positions
job = f'samtools index {out_prefix}.bam'
print(f'job: {job}')
!time {job}

job: samtools index ctrl.bam

real	0m1.259s
user	0m1.244s
sys	0m0.012s


In [8]:
# create a BAM file from reads mapped to chrA only
job = f'samtools view -b {out_prefix}.bam chrA > {out_prefix}A.bam'
print(f'job: {job}')
!time {job}

# index this BAM file
job = f'samtools index {out_prefix}A.bam'
print(f'job: {job}')
!time {job}

job: samtools view -b ctrl.bam chrA > ctrlA.bam

real	0m0.526s
user	0m0.491s
sys	0m0.012s
job: samtools index ctrlA.bam

real	0m0.089s
user	0m0.082s
sys	0m0.004s


In [9]:
# generate BigWig files from BAM files
job = f'/usr/local/deeptools3/bin/bamCoverage -p {threads} -b {out_prefix}.bam -o {out_prefix}.bw'
print(f'job: {job}')
!time {job}

job: /usr/local/deeptools3/bin/bamCoverage -p 1 -b ctrl.bam -o ctrl.bw
bamFilesList: ['ctrl.bam']
binLength: 50
numberOfSamples: None
blackListFileName: None
skipZeroOverZero: False
bed_and_bin: False
genomeChunkSize: None
defaultFragmentLength: read length
numberOfProcessors: 1
verbose: False
region: None
bedFile: None
minMappingQuality: None
ignoreDuplicates: False
chrsToSkip: []
stepSize: 50
center_read: False
samFlag_include: None
samFlag_exclude: None
minFragmentLength: 0
maxFragmentLength: 0
zerosToNans: False
smoothLength: None
save_data: False
out_file_for_raw_data: None
maxPairedFragmentLength: 1000

real	0m12.982s
user	0m11.926s
sys	0m0.140s


In [10]:
# list all BigWig files:
!ls -lh *.bw

-rw-r--r-- 1 boycem4 GEU3302521 712K Dec  3 11:26 ctrl.bw
